In [1]:
import cnvpytor
import subprocess

print(cnvpytor.__version__)

1.3.1


In [2]:
filename = "test_with_del__2_formated.fa"
command = f"cat {filename} | wc -l"
result = subprocess.check_output(command, shell=True)
line_count = int(result.strip())
line_count

142635

In [3]:
read_count = line_count * 50 * 30 / 202
read_count = round(read_count)
read_count

1059171

In [4]:
command = f"wgsim -d 300 -N {read_count} -1 101 -2 101 {filename} frag1Copy_test.fastq frag2Copy_test.fastq"
result = subprocess.check_output(command, shell=True)
print(result)

[wgsim] seed = 1700512266
[wgsim_core] calculating the total length of the reference sequence...
[wgsim_core] 1 sequences, total length: 7131684


b'chr22\t435\tT\t-\t-\nchr22\t571\tT\tG\t-\nchr22\t2103\tT\tY\t+\nchr22\t3261\tC\tY\t+\nchr22\t4180\tG\tC\t-\nchr22\t4466\tC\tA\t-\nchr22\t4871\tG\tA\t-\nchr22\t5064\tG\tS\t+\nchr22\t5222\tC\tS\t+\nchr22\t5749\tG\tS\t+\nchr22\t7253\tC\tY\t+\nchr22\t7658\tT\tC\t-\nchr22\t7698\tC\tS\t+\nchr22\t7791\tT\t-\t-\nchr22\t8813\tT\tY\t+\nchr22\t9923\tG\tS\t+\nchr22\t12108\tG\t-\t-\nchr22\t12580\tT\tK\t+\nchr22\t13188\tT\tY\t+\nchr22\t15153\tG\tK\t+\nchr22\t15661\tG\t-\t-\nchr22\t16934\tT\tA\t-\nchr22\t17579\tC\tS\t+\nchr22\t20254\t-\tA\t+\nchr22\t21954\tG\tA\t-\nchr22\t22060\tG\tK\t+\nchr22\t24148\tG\tR\t+\nchr22\t26527\tT\tG\t-\nchr22\t31223\tT\tW\t+\nchr22\t31852\tA\tC\t-\nchr22\t34806\tA\tW\t+\nchr22\t37106\tG\t-\t-\nchr22\t37203\tA\t-\t-\nchr22\t38980\tT\tY\t+\nchr22\t39793\tT\tY\t+\nchr22\t40090\tA\tR\t+\nchr22\t40373\tC\tS\t+\nchr22\t40475\tA\tR\t+\nchr22\t42469\tC\tT\t-\nchr22\t42880\tT\tG\t-\nchr22\t43328\tT\t-\t-\nchr22\t45102\tA\tM\t+\nchr22\t47326\tT\tY\t+\nchr22\t51674\tG\tR\t+\nchr2

In [5]:
# bowtie related stuff
!bowtie2-build reference.fa bias
!bowtie2 -p 4 -k 15 --no-mixed -x bias -1 frag1Copy_test.fastq -2 frag2Copy_test.fastq -S biasOut.sam

Settings:
  Output files: "bias.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  reference.fa
Building a SMALL index
Reading reference sizes
  Time reading reference sizes: 00:00:01
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
bmax according to bmaxDivN setting: 12826141
Using parameters --bmax 9619606 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 9619606 --dcv 1024
Constructing suffix-array element

In [6]:
# sam to bam file conversion
!samtools sort biasOut.sam -o biasOut_sorted.bam
!samtools index biasOut_sorted.bam

[bam_sort_core] merging from 1 files and 1 in-memory blocks...


In [7]:
app = cnvpytor.Root("test.pytor", create=True, max_cores=8)

In [8]:
app.rd(["biasOut_sorted.bam"])

In [9]:
app.calculate_histograms([100, 1000, 10000, 100000, 1000000, 10000000])

In [10]:
app.partition([100, 1000, 10000, 100000, 1000000, 10000000])

In [11]:
calls = app.call([100, 1000, 10000, 100000, 1000000, 10000000], print_calls=True)

duplication	chr22:643201-644800	1600	1.4844	7.470587e-06	3.283714e-61	1.000000e+00	1.000000e+00	0.0000	1.0000	0
duplication	chr22:1306101-1325100	19000	1.3961	0.000000e+00	2.129196e-275	0.000000e+00	1.724318e-245	0.0000	1.0000	0
duplication	chr22:2926201-2963000	36800	3.1299	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:4546401-4713300	166900	1.6645	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:6655001-6661800	6800	2.4267	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:6847801-6854600	6800	1.6263	0.000000e+00	1.569911e-02	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:8039401-8047900	8500	1.9422	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:9424201-9461600	37400	1.4863	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:13230601-13395900	165300	1.4062	0.000000e+00	0.000000e+00	0.00000

In [12]:
for bin_size in calls:
    print("Bin size: ", bin_size)

Bin size:  100
Bin size:  1000
Bin size:  10000
Bin size:  100000
Bin size:  1000000
Bin size:  10000000


In [13]:
print(f"{filename=}, Histogram Size: 100")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[100]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__2_formated.fa', Histogram Size: 100
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,643201,644800,1600,1.4843863594929578,0.0
duplication,chr22,1306101,1325100,19000,1.3961019193371382,0.0
duplication,chr22,2926201,2963000,36800,3.1299113374806655,0.0
duplication,chr22,4546401,4713300,166900,1.6644720773682644,0.0
duplication,chr22,6655001,6661800,6800,2.4266689316383574,0.0
duplication,chr22,6847801,6854600,6800,1.626268125179751,0.0
duplication,chr22,8039401,8047900,8500,1.942234472059516,0.0
duplication,chr22,9424201,9461600,37400,1.4862710936540764,0.0
duplication,chr22,13230601,13395900,165300,1.4061678272183915,0.0
duplication,chr22,13449001,13453700,4700,1.3228603391063758,0.0
duplication,chr22,13480801,13512800,32000,4.225723451201771,0.0
duplication,chr22,15242601,15246400,3800,3.0152341416348523,0.0
duplication,chr22,18662501,18664600,2100,1.4599688472495447,0.0
duplication,chr22,18664801,18665900,1100,1.4978934817277458,0.0
duplica

In [14]:
print(f"{filename=}, Histogram Size: 1000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[1000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__2_formated.fa', Histogram Size: 1000
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,1306001,1325000,19000,1.3957121456043917,0.0
duplication,chr22,2926001,2963000,37000,3.1198304215142616,0.0
duplication,chr22,4546001,4713000,167000,1.6639440514135981,0.0
duplication,chr22,6655001,6662000,7000,2.391623502570917,0.0
duplication,chr22,8040001,8048000,8000,1.946523263311974,0.0
duplication,chr22,9424001,9461000,37000,1.4866269764754068,0.0
duplication,chr22,13231001,13396000,165000,1.4061899305321943,0.0
duplication,chr22,13481001,13513000,32000,4.222530398168489,0.0
duplication,chr22,18705001,18881000,176000,2.7507954305434845,0.0001409637257045689
duplication,chr22,19259001,19268000,9000,3.2542526652454162,0.0
duplication,chr22,21248001,21382000,134000,1.4947669292321053,0.00012675425914116068
duplication,chr22,21533001,21548000,15000,1.3054025182093958,0.0026642496154691275
duplication,chr22,21577001,21624000,47000,1.5559139349933417,0.00013

In [15]:
print(f"{filename=}, Histogram Size: 10,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[10000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__2_formated.fa', Histogram Size: 10,000
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,4550001,4710000,160000,1.6573216616640443,0.0
duplication,chr22,6570001,7010000,440000,1.0855994455276086,0.0
duplication,chr22,13230001,13420000,190000,1.3539620601864157,0.0
duplication,chr22,18710001,18890000,180000,2.670816835156934,0.00016275270562438794
duplication,chr22,21250001,21380000,130000,1.4741515370354976,0.00012751399262589294
duplication,chr22,21490001,21640000,150000,1.3219568305447793,0.0004201957064859154
duplication,chr22,24200001,24240000,40000,2.5082759022050665,4.7036442790619886e-05
duplication,chr22,25150001,25310000,160000,1.277054734732412,0.00012919104223242122
duplication,chr22,28470001,28600000,130000,3.080092471163881,4.274700529649185e-05
duplication,chr22,30800001,30980000,180000,2.906119616062182,2.59848638168267e-05
duplication,chr22,31150001,31360000,210000,1.4261199266844138,4.966088708533159e-05
duplication,chr22,31870

In [16]:
print(f"{filename=}, Histogram Size: 100,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[100000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__2_formated.fa', Histogram Size: 100,000
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,3000001,6900000,3900000,1.0726155634564025,0.0
duplication,chr22,47100001,51400000,4300000,1.4079806448562475,7.527331741553581e-05


In [17]:
print(f"{filename=}, Histogram Size: 1,000,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[1000000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__2_formated.fa', Histogram Size: 1,000,000
type,chromosome_name,start,end,size,cnv_level,q0


In [18]:
print(f"{filename=}, Histogram Size: 10,000,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[10000000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__2_formated.fa', Histogram Size: 10,000,000
type,chromosome_name,start,end,size,cnv_level,q0


In [19]:
del app